# Libraries and Data

In [1]:
#change directory
%cd /content/drive/MyDrive/Time Series Forecasting Product

/content/drive/MyDrive/Time Series Forecasting Product


In [2]:
!pip install darts

In [3]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid

In [4]:
#load the data
#YYYY-MM-DD
df = pd.read_csv('nyc_data.csv', index_col = 0, parse_dates = True)
df.head()

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077


In [5]:
#Rename variable
df = df.rename(columns = {'Demand': 'y'})
df.head(0)

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,


# Prepare for LSTM

In [6]:
# Creating Time Series
from darts.timeseries import TimeSeries
series = TimeSeries.from_series(df.y)
covariates = TimeSeries.from_dataframe(df.iloc[:,1:])

In [7]:
# Seasonality / trend
from darts.utils.timeseries_generation import datetime_attribute_timeseries

# Year
year_series = datetime_attribute_timeseries(
    pd.date_range(start = series.start_time(),
                  freq = series.freq_str,
                  periods = df.shape[0]),
    attribute = "year",
    one_hot = False)

# Month
month_series = datetime_attribute_timeseries(year_series,
                                             attribute= "month",
                                             one_hot = True)

# Month
weekday_series = datetime_attribute_timeseries(year_series,
                                             attribute= "weekday",
                                             one_hot = True)


In [8]:
# Scaling
from darts.dataprocessing.transformers import Scaler
transformer1 = Scaler()
transformer2 = Scaler()

In [9]:
# Scaling Y
y_transformed = transformer1.fit_transform(series)

In [10]:
# Stacking the covariates
covariates = covariates.stack(year_series)

In [11]:
# Scaling the Covariates
covariates_transformed = transformer2.fit_transform(covariates)
covariates_transformed

<TimeSeries (DataArray) (Date: 2192, component: 6, sample: 1)>
array([[[0.        ],
        [0.        ],
        [0.        ],
        [0.31522013],
        [0.06072561],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.34163522],
        [0.38865336],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.4045283 ],
        [0.50172739],
        [0.        ]],
...
       [[0.        ],
        [0.        ],
        [0.        ],
        [0.31018868],
        [0.49869401],
        [1.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.37283019],
        [0.56324495],
        [1.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.42943396],
        [0.4527611 ],
        [1.        ]]])
Coordinates:
  * Date       (Date) datetime64[ns] 2015-01-01 2015-01-02 ... 2020-12-31
  * component  (component) object 'Easter' 'Thanksgiving' ... 'Marketing' 'year'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [12]:
# Stack the Seasonal variables
covariates_transformed = covariates_transformed.stack(month_series)
covariates_transformed = covariates_transformed.stack(weekday_series)
covariates_transformed

<TimeSeries (DataArray) (Date: 2192, component: 25, sample: 1)>
array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
...
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]])
Coordinates:
  * Date       (Date) datetime64[ns] 2015-01-01 2015-01-02 ... 2020-12-31
  * component  (component) object 'Easter' 'Thanksgiving' ... 'weekday_7'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

# LSTM Model

In [13]:
# Build the LSTM model
# https://unit8co.github.io/darts/generated_api/darts.models.forecasting.rnn_model.html
from darts.models import RNNModel
model = RNNModel(model = "LSTM",
                 hidden_dim = 20,
                 n_rnn_layers = 2,
                 dropout = 0.2,
                 n_epochs = 20,
                 optimizer_kwargs = {"lr": 0.003},
                 random_state = 1502,
                 training_length = 20,
                 input_chunk_length=15,
                 pl_trainer_kwargs = {"accelerator": "gpu", "devices": [0]}
                 )

In [14]:
# Fit the model to the data
model.fit(y_transformed,
          future_covariates = covariates_transformed)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 7.2 K 
4 | V             | Linear           | 21    
---------------------------------------------------
7.2 K     Trainable params
0         Non-trainable params
7.2 K     Total params
0.029     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


RNNModel(model=LSTM, hidden_dim=20, n_rnn_layers=2, dropout=0.2, training_length=20, n_epochs=20, optimizer_kwargs={'lr': 0.003}, random_state=1502, input_chunk_length=15, pl_trainer_kwargs={'accelerator': 'gpu', 'devices': [0]})

# Cross-Validation

In [15]:
# CV
cv = model.historical_forecasts(series = y_transformed,
                                future_covariates = covariates_transformed,
                                start = df.shape[0] - 180,
                                forecast_horizon = 31,
                                stride = 16,
                                retrain = True,
                                last_points_only = False)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 7.2 K 
4 | V             | Linear           | 21    
---------------------------------------------------
7.2 K     Trainable params
0         Non-trainable params
7.2 K     Total params
0.029     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params
-

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params
-

...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params
-

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params
-

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [16]:
from sklearn.metrics import mean_squared_error

# Store the results
rmse_cv = []

for i in range(len(cv)):

  # Compute the RMSE for the CV
  predictions = TimeSeries.pd_series(transformer1.inverse_transform(cv[i]))

  # Actual values
  start = predictions.index.min()
  end = predictions.index.max()
  actuals = df.y[start:end]

  #compute the error
  error_cv = mean_squared_error(actuals, predictions, squared = False)

  # save the error
  rmse_cv.append(error_cv)

print(f"The RMSE is {np.mean(rmse_cv )}")

The RMSE is 115.85711415220798


# Parameter Tuning

In [17]:
# Grid
param_grid = {'n_rnn_layers': [1,2],
              'hidden_dim': [10,20],
              'dropout': [0.1,0.2],
              'n_epochs': [10,20],
              'lr': [0.003],
              'training_length': [ 20],
              'input_chunk_length': [15]}
grid = ParameterGrid(param_grid)
len(list(grid))

16

In [ ]:
# Parameter Tuning
rmse = []

# Loop
for params in grid:
  # Build the LSTM model
  model = RNNModel(model = "LSTM",
                  hidden_dim = params['hidden_dim'],
                  n_rnn_layers = params['n_rnn_layers'],
                  dropout = params['dropout'],
                  n_epochs = params['n_epochs'],
                  optimizer_kwargs = {"lr": 0.003},
                  random_state = 1502,
                  training_length = 20,
                  input_chunk_length=15,
                  pl_trainer_kwargs = {"accelerator": "gpu", "devices": [0]}
                  )

  # Fit the model to the data
  model.fit(y_transformed,
            future_covariates = covariates_transformed)

  # CV
  cv = model.historical_forecasts(series = y_transformed,
                                  future_covariates = covariates_transformed,
                                  start = df.shape[0] - 180,
                                  forecast_horizon = 31,
                                  stride = 16,
                                  retrain = True,
                                  last_points_only = False)

  # Measure and store the error
  # Store the results
  rmse_cv = []

  for i in range(len(cv)):

    # Compute the RMSE for the CV
    predictions = TimeSeries.pd_series(transformer1.inverse_transform(cv[i]))

    # Actual values
    start = predictions.index.min()
    end = predictions.index.max()
    actuals = df.y[start:end]

    #compute the error
    error_cv = mean_squared_error(actuals,
                                  predictions,
                                  squared = False)

    # save the error
    rmse_cv.append(error_cv)

  error = np.mean(rmse_cv)
  rmse.append(error)

In [24]:
# Parameter Tuning outcome
tuning_results = pd.DataFrame(grid)
tuning_results['rmse'] = rmse
tuning_results

,dropout,hidden_dim,input_chunk_length,lr,n_epochs,n_rnn_layers,training_length,rmse
0,0.1,10,15,0.003,10,1,20,158.387555
1,0.1,10,15,0.003,10,2,20,193.268426
2,0.1,10,15,0.003,20,1,20,182.027918
3,0.1,10,15,0.003,20,2,20,124.044204
4,0.1,20,15,0.003,10,1,20,95.828042
5,0.1,20,15,0.003,10,2,20,113.648863
6,0.1,20,15,0.003,20,1,20,99.906600
7,0.1,20,15,0.003,20,2,20,120.200227
8,0.2,10,15,0.003,10,1,20,158.387555
9,0.2,10,15,0.003,10,2,20,192.279664


In [25]:
# Exporting the tuned parameters
best_params = tuning_results[tuning_results.rmse == tuning_results.rmse.min()].transpose()
best_params

dropout                0.100000
hidden_dim            20.000000
input_chunk_length    15.000000
lr                     0.003000
n_epochs              10.000000
n_rnn_layers           1.000000
training_length       20.000000
rmse                  95.828042
Name: 4, dtype: float64

In [26]:
# Isolate the params
n_rnn_layers = int(best_params.loc['n_rnn_layers'])
hidden_dim = int(best_params.loc['hidden_dim'])
dropout = float(best_params.loc['dropout'])

# Parameter Tuning part 2

In [27]:
# Grid
param_grid = {'n_rnn_layers': [n_rnn_layers],
              'hidden_dim': [hidden_dim],
              'dropout': [dropout],
              'n_epochs': [10,20],
              'lr': [0.001, 0.003],
              'training_length': [20, 30],
              'input_chunk_length': [15, 20]}
grid = ParameterGrid(param_grid)
len(list(grid))

16

In [ ]:
# Parameter Tuning
rmse = []

# Loop
for params in grid:
  # Build the LSTM model
  model = RNNModel(model = "LSTM",
                  hidden_dim = params['hidden_dim'],
                  n_rnn_layers = params['n_rnn_layers'],
                  dropout = params['dropout'],
                  n_epochs = params['n_epochs'],
                  optimizer_kwargs = {"lr": params['lr']},
                  random_state = 1502,
                  training_length = params['training_length'],
                  input_chunk_length=params['input_chunk_length'],
                  pl_trainer_kwargs = {"accelerator": "gpu", "devices": [0]}
                  )

  # Fit the model to the data
  model.fit(y_transformed,
            future_covariates = covariates_transformed)

  # CV
  cv = model.historical_forecasts(series = y_transformed,
                                  future_covariates = covariates_transformed,
                                  start = df.shape[0] - 180,
                                  forecast_horizon = 31,
                                  stride = 16,
                                  retrain = True,
                                  last_points_only = False)

  # Measure and store the error
  # Store the results
  rmse_cv = []

  for i in range(len(cv)):

    # Compute the RMSE for the CV
    predictions = TimeSeries.pd_series(transformer1.inverse_transform(cv[i]))

    # Actual values
    start = predictions.index.min()
    end = predictions.index.max()
    actuals = df.y[start:end]

    #compute the error
    error_cv = mean_squared_error(actuals,
                                  predictions,
                                  squared = False)

    # save the error
    rmse_cv.append(error_cv)

  error = np.mean(rmse_cv)
  rmse.append(error)

In [29]:
# Parameter Tuning outcome
tuning_results = pd.DataFrame(grid)
tuning_results['rmse'] = rmse
tuning_results

,dropout,hidden_dim,input_chunk_length,lr,n_epochs,n_rnn_layers,training_length,rmse
0,0.1,20,15,0.001,10,1,20,190.397659
1,0.1,20,15,0.001,10,1,30,182.814111
2,0.1,20,15,0.001,20,1,20,110.841949
3,0.1,20,15,0.001,20,1,30,118.214680
4,0.1,20,15,0.003,10,1,20,95.828042
5,0.1,20,15,0.003,10,1,30,126.807636
6,0.1,20,15,0.003,20,1,20,99.906600
7,0.1,20,15,0.003,20,1,30,108.100203
8,0.1,20,20,0.001,10,1,20,190.519771
9,0.1,20,20,0.001,10,1,30,183.040097


In [30]:
# Exporting the tuned parameters
best_params = tuning_results[tuning_results.rmse == tuning_results.rmse.min()].transpose()
best_params.to_csv("Forecasting Product/best_params_lstm.csv")